In [1]:
import pandas as pd
from ntscraper import Nitter

C:\Users\bookz\AppData\Local\Temp\ipykernel_9708\3136252630.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
Scraper = Nitter()

Testing instances: 100%|██████████| 77/77 [01:47<00:00,  1.39s/it]


In [3]:
#tweets =Scraper.get_tweets("realDonaldTrump",mode ='user',number = 100)
def scraping(input):
    tweets =Scraper.get_tweets(input,mode ='user',number = 500)
    return tweets

tweets = scraping("elonmusk")

24-Apr-24 04:15:43 - No instance specified, using random instance https://nitter.privacydev.net
24-Apr-24 04:15:49 - Current stats for elonmusk: 20 tweets, 0 threads...
24-Apr-24 04:15:53 - Current stats for elonmusk: 40 tweets, 0 threads...
24-Apr-24 04:15:56 - Current stats for elonmusk: 60 tweets, 0 threads...
24-Apr-24 04:16:00 - Current stats for elonmusk: 80 tweets, 0 threads...
24-Apr-24 04:16:04 - Current stats for elonmusk: 100 tweets, 0 threads...
24-Apr-24 04:16:08 - Current stats for elonmusk: 120 tweets, 0 threads...
24-Apr-24 04:16:11 - Fetching error: Instance has been rate limited.Use another instance or try again later.


In [4]:
final_tweets = []
for tweet in tweets['tweets']:
    #data = [tweet['link'],tweet['text'],tweet['date'],tweet['stats']['likes'],tweet['stats']['comments']]
    data = [tweet['text'],tweet['date']]
    final_tweets.append(data)

In [5]:
data =pd.DataFrame(final_tweets, columns= ['text','date'])
#columns= ['link','text','date','no_likes','comment'])

In [6]:
data['date'] = pd.to_datetime(data['date'], format='%b %d, %Y · %I:%M %p %Z')
data['date'] = data['date'].dt.to_period('M')


C:\Users\bookz\AppData\Local\Temp\ipykernel_9708\4118179080.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['date'] = data['date'].dt.to_period('M')


In [7]:
data.tail()

,text,date
115,NPR has become a hard left propaganda machine ...,2024-04
116,FSD Supervised continues to improve with every...,2024-04
117,Interesting series about a potentially good fu...,2024-04
118,United States laws prevent 𝕏 from participatin...,2024-04
119,Interesting,2024-04


In [8]:
import dill
def clean(text):
    import re
    import nltk
    from nltk.util import pr
    stemmer = nltk.SnowballStemmer("english")
    from nltk.corpus import stopwords
    import string
    stopword = set(stopwords.words("english"))
    text = str(text).lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

with open('../backend/textback/models/clean_function.pkl', 'wb') as f:
     dill.dump(clean, f)

<>:12: SyntaxWarning: invalid escape sequence '\S'
<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:12: SyntaxWarning: invalid escape sequence '\S'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\bookz\AppData\Local\Temp\ipykernel_9708\3327462015.py:12: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\bookz\AppData\Local\Temp\ipykernel_9708\3327462015.py:16: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w\d\w', '', text)


In [9]:
import pickle
model = pickle.load(open("../backend/textback/models/model.pkl","rb"))
cv = pickle.load(open("../backend/textback/models/vectorizer.pkl","rb"))

def predict(input):
    input = cv.transform([input]).toarray()
    predict = model.predict(input)[0]
    return predict

In [10]:
tweets_cleaned = []
Negative = 0
Nuetral = 0
for i in range(len(final_tweets)):
    clean_tweets = clean(final_tweets[i])
    if clean_tweets != "":
        tweets_cleaned.append(clean_tweets)


for i in range(len(tweets_cleaned)):
    prediction = predict(tweets_cleaned[i])
    if prediction == "No hate and offensive speech":
        Nuetral += 1
    elif prediction == "Offensive language detected":
        Negative += 1
    else :
       Negative += 1

#keep the score for each text in the dataframe file
data['score'] = data['text'].apply(predict)

#change score to numeric form 0,1,2
data['score'] = data['score'].replace({'No hate and offensive speech':0,'Offensive language detected':1,'Hate speech Detected':2})
print("Score about Negative and Nuetral in This user is\n Negative = " , Negative , "\n Nuetral = " , Nuetral)


Score about Negative and Nuetral in This user is
 Negative =  18 
 Nuetral =  102


C:\Users\bookz\AppData\Local\Temp\ipykernel_9708\3027145680.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['score'] = data['score'].replace({'No hate and offensive speech':0,'Offensive language detected':1,'Hate speech Detected':2})


In [11]:
#data to csv file
data['Month'] = data['date'].dt.month
data.to_csv('gay.csv',index=False)